## PIO
### Projekt 5 [50 pkt]

Proszę użyć tego arkusza do programowania. Na początek proszę zmienić 'nazwisko_imie' na Państwa nazwisko i imie, bez polskich znaków diakrytycznych.

Jak już Państwo zaprogramują wszystko, proszę zapisać arkusz (File->Save and Checkpoint) oraz zapisać arkusz na dysk (File->Download as->Notebook). Ten plik proszę wgrać na classroom. 

### Automatyczna klasówka z języka angielskiego

Proszę napisać program generujący automatyczne klasówki z języka angielskiego. Bazujemy na problemie z lab 9, ale treści do klasówek proszę pobierać ze strony interentowej oferującej naukę języka np: https://www.gettinenglish.com

Funkcjonalność 
1. Uczniowie - czytanie listy uczniów z pliku
2. Klasówka
    1. minimum 3 zadania różniące się choć trochę treścią dla każdego z uczniów
    2. zapisywanie treści klasówek do plików imiennych
    3. składowanie plików w katalogu o nazwie tożsamej z nazwą egzaminu (klasówki...)
3. Pobieranie treści z www
    1. konfiguracja adresu strony www powinna być w miarę prosta i zrozumiała
    2. parsowanie strony powinno być choć w niewielkim stopniu dostępne za pomocą parametrów (np: klasy tagów do których się odwołuje parser)

Opcjonalnie
* automatyczne wysyłanie klasówek na adresy mailowe uczniów


In [51]:
from ipywidgets import *
from IPython.display import clear_output, display
from tkinter import Tk, filedialog
import pandas as pd

df = pd.DataFrame()

def display_rows(rows):
    lista.clear_output()
    for key, value in rows:
        inbox = [Label(value='#'+str(key), layout={'width' : '60px'})]
        inbox.append(Label(value=str(value[0]), layout={'width' : '80px'}, disabled=True))
        inbox.append(Label(value=str(value[1]), layout={'width' : '80px'}, disabled=True))
#         inbox.append(Text(value=str(value[2]), layout={'width' : '10%'}, disabled=True))
        inbox.append(Label(value=str(value[2]), layout={'width' : '250px'}, disabled=True))
        inbox.append(Checkbox(value=False, layout={'width' : '15%'}, disabled=False, indent=False))
        box = widgets.HBox(inbox)
        with lista:
            display(box)

def open_file(file):
    fob=open(file,'r')
    df = pd.read_csv(fob, index_col="ID")                                         
    display_rows(df.iterrows())
    
        
            
def select_files(b):
    clear_output()                                         
    root = Tk()
    root.withdraw()                                        
    root.call('wm', 'attributes', '.', '-topmost', True)   
    b.files = filedialog.askopenfilename(multiple=True, title = "Select student list file",filetypes = (("CSV files","*.csv"),("TXT files","*.txt"),))    # List of selected files will be set button's file attribute.
    for file in b.files:
        open_file(file)
        
        
        
fileselect = Button(description="Load student lists")
fileselect.on_click(select_files)
addstudent = Button(description="Add student")
addstudent.on_click(select_files)
savelist = Button(description="Save list")
savelist.on_click(select_files)
selectall = Button(description="Select all")
selectall.on_click(select_files)

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='none')


lista = Output(layout=box_layout)

hbox =  HBox(children=[fileselect,addstudent,savelist, selectall])
vbox =  VBox(children=[hbox,lista])
grid = Box(children=[vbox],
                      layout=Layout(border='1px solid black',
                            width='100%',
                            height='400px'))
display(grid)
open_file('DOWNS/student lists/students.csv')

Box(children=(VBox(children=(HBox(children=(Button(description='Load student lists', style=ButtonStyle()), But…

In [52]:
import urllib.request as urllib2
from bs4 import BeautifulSoup

def parseGettinEnglish(url, _type='RC', log = False):
    """ 
    gettinenglish.com parser. 
        RC = Reading Comprehension 
    """
    data = urllib2.urlopen(url)
    soup = BeautifulSoup(data, 'html.parser')


    title = ''
    stuff = ''
    answers = ''
    try:
        title = soup.find("div", {"class": "single-post-content"}).h2.extract().getText().strip()
    except:
        title = soup.find("h1", {"class": "single-post-title"}).getText().strip()

    stuff = soup.find("div", {"class": "single-post-content"}).findAll('p')
    
    for n, p in enumerate(stuff):
        if log:
            print(n)  
            print(p.prettify())
            print()
        try:
    #         p.a.replace_with('')
            p.a.extract()
            p.div.extract()
            pass

        except:
            pass

        if (p.getText()=='') or (p.strong is not None):
            stuff.pop(n)

    text = '\n\n'.join((map(BeautifulSoup.getText, stuff)))
    
    print(url)
    print(title) 
    print(text)
    print()
    return (title, text)


url = "https://www.gettinenglish.com/egzaminy/czytanie-ze-zrozumieniem/"

data = urllib2.urlopen(url)
soup = BeautifulSoup(data, 'html.parser')
_urls = [x['href'] for x in soup.find("div", {"class": "main-section"}).findAll("a", {"class": "post-url"})]

for n, _ in enumerate(_urls):
    (title, text) = parseGettinEnglish(url = _,  _type='RC')
    print(n, _)

https://www.gettinenglish.com/materialy-do-nauki-angielskiego/
I wanna stay at home
Agoraphobia is an anxiety disorder in which people may experience fear and frequently refrain from getting into a situation that can make them feel trapped, embarrassed or helpless. Some agoraphobics are fearful of open spaces. Some even in a medical emergency, refuse to leave their homes because stepping outside their comfort zone is overwhelming. An agoraphobic also breaks out in a sweat while standing in a queue or being in a crowd. Most people who have agoraphobia avoid the place where deep emotional distress is likely to occur; they feel that they need a chum to accompany them to any public places.

„My name’s Melissa and I’ve suffered from agoraphobia since my early twenties. I know it sounds outlandish but every time I wanna go out I feel tingling in my hands and feet, nausea, and a sense of impending doom!”

A number of physical and environmental factors can trigger agoraphobia and other phobias

In [53]:
from ipywidgets import *
import validators
import urllib.request as urllib2
from bs4 import BeautifulSoup, NavigableString, Tag
import re 

class GettinParser():  
    _gui = widgets.Output(layout={'border': '1px solid black'})
    _url = None
    _soup = BeautifulSoup()
    _title_filter = ['post-header-title']
    
    
    def display_GUI(self):
#         self._gui  = widgets.Output(layout={'border': '1px solid black'})
        self._gui.clear_output()
        self.updateGUI()
        display(self._gui)
        
        
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        try:
            if validators.url(value):
                print("Setting value of url to {}".format(value))
                self._url = value
                self.update()
            else:
                print("Error: {} is not a valid url".format(value))
            
        except Exception as e:
            print(f"{type(e).__name__} was raised: {e}") 
    
    def update(self):
        self.parseURL()

    def parseURL(self):
        data = urllib2.urlopen(self._url)
        self._soup = BeautifulSoup(data, 'html.parser')
        self._titleOutput=widgets.Output()
        
        def soupWalker(soup, level=0):
       
            indent = (level) * '·' + 'o'

            if soup.name is not None:
                for child in soup.children:
                    if isinstance(child, Tag):
                        name = child.name
                        classList = (child.get('class'))
                        if classList is not None:
                            classes = ', '.join(list(classList))
                        else:
                            classes = 'none'
                        with self._titleOutput:
                            print(indent, '<{}> ({})'.format(name, classes))

                    soupWalker(child, level+1)
                    
        self._titles = self._soup.find_all(class_=self._title_filter)
        
#         self._titles = self._soup.find_all(class_=lambda c: all(f in c for f in self._title_filter))

        
        for node in (self._titles):
            soupWalker(node.parent)
            
        self.updateGUI()
        
    def updateGUI(self):
        self._gui.clear_output()
        
        buttonLayout={'width': '120px'}
        textAreaLayout={'width': '50%', 'height': '300px'}
        
        urlText = (Text(value=self.url,
                        placeholder='Enter url...', 
                        disabled=False, 
                        continuous_update=False))
        
        def urlButtonCallback(b, value=None):
            self.url = (urlText.value) 
            
        urlText.observe(urlButtonCallback, names='value')
        
        parseText = (Textarea(value=self._soup.prettify(),
                        disabled=False,  
                        layout=textAreaLayout))
        
        titleText = (Text(value=', '.join(self._title_filter),
                          description='Filter classes:', 
                          continuous_update=False))
        
        def on_value_change(change):
            self._title_filter=(change['new']).split(',')
            print(self._title_filter)
            self.update()

        titleText.observe(on_value_change, names='value')
        
        titleDropdown = Dropdown(options=list(map(BeautifulSoup.getText, self._titles)),
                                description='Choose:', 
                                disabled=False)
        
        titleOutput= self._titleOutput
        
        
        app = VBox(children = [urlText,
                              HBox(children = [VBox(children = [titleText, titleDropdown, titleOutput]), 
                                               parseText])]) #titleOutpu
        
        with self._gui:
            display(app)
        

In [54]:
from ipywidgets import *
import validators
import urllib.request as urllib2
from bs4 import BeautifulSoup, NavigableString, Tag

def soupWalker(soup, level=0, output=None):
    if (output is None):
        output=widgets.Output()
        
    indent = (level) * ' ' # + 'o' # ·

    if soup.name is not None:
        for child in soup.children:
            if isinstance(child, Tag):
                name = child.name
                ID = ''
                classes = ''
                try:
                    classes = '.'+'.'.join(list(child.get('class')))
                    ID = '#'+(child.get('id'))
                except Exception as e:
                    handleException(e)
                with output:
                    print('{}{}{}{}'.format(indent, name, ID, classes))

            soupWalker(child, level+1, output)
    return output
            
def url2soup(url):
    data = urllib2.urlopen(url)
    return BeautifulSoup(data, 'html.parser')

def soupSelector(soup, selector):
    return soup.select(selector)

def handleException(e, log=False):
    if log:
        print(f"{type(e).__name__} was raised: {e}") 
    else:
        pass
    
class parserGUI(object):
    _parent = None
    _nodeTree= None 
    
    urlText = Text(value='',
                    placeholder='Enter url...', 
                    disabled=False, 
                    continuous_update=False,
                   layout={'width': '100%'}
                   )
    
    
    assignDropdown = Dropdown(options=['title','body','questions'],
                              disabled=False,
                              layout={'width': 'max-content'},
                             )
    
    SelectorLayout = Layout(width='100%')
    OutputLayout = Layout(width='100%', height='300px', border='1px solid lightgray')
    cssSelectorLabel1 = Button(description='Title selector:', button_style='primary')
    
    cssSelectorText1 = Text(value='',
                            placeholder='css selector', 
                            disabled=False, 
                            continuous_update=False,  
                            layout=SelectorLayout
                           )
    
    nodeTreeOutput =  Output(layout=OutputLayout)
    
    resultText = Textarea(value='',
                          placeholder='OUTPUT', 
                          disabled=False, 
                          continuous_update=False,  
                            layout={'width': '95%'}
                         )
    
    loadTemplateButton = Button(description="Load")
    
    saveTemplateButton = Button(description="Save template")
    
    categoryText = Text(value='',
                         placeholder='category...', 
                         disabled=False, 
                         continuous_update=False
                        )
    
    addToDataBase = Button(description="Add to database", 
                           disabled=True, 
                           layout = {'width':'100%'})

    app=VBox([HBox([urlText, loadTemplateButton, saveTemplateButton]),
              HBox([cssSelectorLabel1, cssSelectorText1]),
              nodeTreeOutput,
              resultText, 
              HBox([ assignDropdown , categoryText]), 
              addToDataBase
             ],
            layout = Layout(display='flex', flex_flow='column', align_items='stretch', justify_content='center',width='100%'))
    
    
    _widgetsDict = {'url':urlText, 'selector1':cssSelectorText1,
                    'nodeTree':nodeTreeOutput,
                    'result':resultText,  'category':categoryText }
    _widgetsList = list(_widgetsDict.values())

    def __init__(self):
        self.setCurrentValues()
        pass
    
    @classmethod
    def setCurrentValues(cls):
        if cls._parent is not None:
            
            if cls._parent.url is not None:
                cls.urlText.value = cls._parent.url 
            
            if cls._parent.selector1 is not None:
                cls.cssSelectorText1.value = cls._parent.selector1 
                
                if cls._parent.soup is not None:
                    cls._nodeTree = soupSelector(cls._parent.soup, cls._parent.selector1)

        if cls._nodeTree is not None:
            cls.nodeTreeOutput.clear_output()
            for node in cls._nodeTree:
                soupWalker(node ,0,cls.nodeTreeOutput)      
        
    @classmethod
    def setParent(cls,value):
        cls._parent=value
        cls.setCurrentValues()
        
    def on_value_change(change): 
        _widgetsKeys = list((parserGUI._widgetsDict.keys()))
        _widgetsList = list(parserGUI._widgetsDict.values())
        index = _widgetsList.index(change.owner)
        key = _widgetsKeys[index]
        
        print(key, change.new)
        
        if key == 'url':
            parserGUI._parent.url = change.new
            
                
        elif key == 'selector1':
            parserGUI._parent.selector1 = change.new
            
            
          
        parserGUI.setCurrentValues()
        
    for _ in _widgetsList:
        _.observe(on_value_change, names='value')
class GettinParser():  
    _gui=parserGUI()
    _url = None
    _soup = None
    _selector1 = 'div.post-header-title span.post-title'
    
    def __init__(self):
        self._gui.setParent (self)
        
    def show(self):
        return (self._gui.app)
    
    def selectDOM(self, selection, returnSoup=True):
        try:
            result = self.soup.select(selection) if returnSoup else '\n'.join(list(map(BeautifulSoup.getText, self.soup.select(selection))))
            
        except Exception as e:
            result = None
            handleException(e)
        
        return result
    
    @staticmethod
    def validateURL(value):
        isValid = validators.url(value)
        if not isValid:
            raise ValueError("Error: {} is not a valid url".format(value))
        return isValid
    
    @staticmethod
    def url2soup(value):
        data = urllib2.urlopen(value)
        result = BeautifulSoup(data, 'html.parser')
        return result

        
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        oldUrl = self._url
        try:
            self.validateURL(value)
            self._url = value
            self._soup = url2soup(value)
            
            self._gui.setCurrentValues()
            
        except Exception as e:
            self.url = oldUrl
            handleException(e)
        
    @property
    def soup(self):
        return self._soup
    
    @soup.setter
    def soup(self, value):
        self._soup =value
        
        
    @property
    def selector1(self):
        return self._selector1
    
    @selector1.setter
    def selector1(self, value):
        selection = self.selectDOM(value)
        self._selector1 = value if selection is not None else None
#         self._gui.nodeTreeOutput1.value=selection
       
         
   
            
        

In [68]:

# soup = url2soup('https://www.gettinenglish.com/czytanki-po-angielsku-b1/')
soup = BeautifulSoup("""<html><body><p class="1 2 3">1</p><span><p>12</p></span></body></html>""", 'html.parser')
soupWalker(soup)

# parserGUI().app
g=GettinParser()
g.url = 'https://www.gettinenglish.com/czytanki-po-angielsku-b1/'

g.show()

In [108]:
import ipywidgets as widgets
import traitlets as tl
import urllib.request as urllib2
from bs4 import BeautifulSoup
import validators

 
class SelectedClass(tl.HasTraits):
    selected = tl.List()
style = {'description_width': 'initial'}

class TabHolder(Tab):
    def __init__(self, tab_objects, tab_names=None):

        if tab_names is None:
            tab_names=range(len(tab_objects))
        if len(tab_names) != len(tab_objects):
            raise ValueError("\n Failed in TabHolder creation, because of lengths of inputs don't match")
        else:
            self.tab_names = list(tab_names)
            self.tab_objects = list(tab_objects)
            super().__init__(children=self.tab_objects)
            for i, title in enumerate(self.tab_names):
                self.set_title(i, title)
         
        
class InputPanelChar(HBox):
    def __init__(self, tab_objects):

        self.tab_objects = list(tab_objects)
        super().__init__(children=self.tab_objects)
        
        
        for w in self.tab_objects:
            print(w.trait_names() )
            pass
#             print(w._model_name )
            
#             print(isinstance(w, ValueWidget) )

#     def get_interact_value(self):
#         return {
#             w.description: w.get_interact_value()
#             for w in self.tab_objects
#         }       
class InputColumnChar(VBox):

    def __init__(self, tab_objects):

        self.tab_objects = list(tab_objects)
        super().__init__(children=self.tab_objects)
        
t1=Text(value='',placeholder='Enter url...', disabled=False, continuous_update=False )  
t2=Text(value='',placeholder='CSS selector...', disabled=False, continuous_update=False )  
h=HTML(value="Hello <b>World</b>")
def transform(value): 
    data = "Hello <b>World</b>"
    if t1.value == '':
        pass
    elif validators.url(t1.value):
        data = urllib2.urlopen(t1.value)
        t2.disabled = False
    else:
        t1.value = ''
        t2.disabled = True
        
    soup = BeautifulSoup(data, 'html.parser')
    sel = t2.value if t2.value != '' else '*'
    try:
        soup = soup.select(sel)
    except:
        t2.value = ''
    
    return str(soup)

tl.directional_link((t1, "value"), (h, "value"), transform=transform)
tl.directional_link((t2, "value"), (h, "value"), transform=transform)
ipc=InputColumnChar([t1, 
                    t2, 
#                     Checkbox(), RadioButtons(options=['pepperoni', 'pineapple', 'anchovies']),
                    h
                   ])


th=TabHolder([ipc]*8, None)
display(th)

WidgetAppender(children=(InputColumnChar(children=(Text(value='', continuous_update=False, placeholder='Enter …